# Keywords in Embedding Space
Import the 3000 entities of books, movies, TV shows and their corresponding keywords

In [5]:
import openai
openai.organization = "REDACTED"
openai.api_key = "REDACTED"

In [9]:
import json
import random
import backoff
with open("data/uri_to_keywords.json", "r") as fp:
    URI_TO_KEYWORDS = json.load(fp)

with open("../multientity-recommender/data/domain.json", "r") as fp:
    domain = json.load(fp)

taste = random.sample(domain.keys(), 10)
keywords = ", ".join([URI_TO_KEYWORDS[uri] for uri in taste])
keywords

/var/folders/hg/vd6q39q15lq3d73cjhq05yk00000gn/T/ipykernel_45081/1736011995.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  taste = random.sample(domain.keys(), 10)


"Blindspot, mystery, crime, drama, action, FBI, Jane Doe, tattoos, memory loss, conspiracy, fast-paced, suspense, thriller, Jaimie Alexander, Sullivan Stapleton, Rob Brown, Martin Gero, Greg Berlanti, Sarah Schechter, New York City, USA, heist, revelations, secrets, teamwork, puzzle solving, amnesia, mind games, crime-solving, tattoo decryption, Cormac McCarthy, No Country for Old Men, 2014, thriller, crime, suspense, neo-Western, Llewelyn Moss, Anton Chigurh, Sheriff Ed Tom Bell, Texas, Mexico, drug deal, moral decay, violence, fate, good vs evil, nihilism, bleak, contemplative, gripping, slow-burn, mature audience, readers who appreciate literary fiction, Tommy Lee Jones, Javier Bardem, Josh Brolin, Kelly Macdonald, Joel and Ethan Coen, United States, Academy Award-winning film, Best Picture, adapted screenplay, Haruki Murakami, A Wild Sheep Chase, The Rat series, surrealism, metaphysical, mystery, Japanese literature, postmodern, existentialism, magical realism, jazz, loneliness, in

In [12]:
@backoff.on_exception(backoff.expo,
                      openai.error.RateLimitError,
                      max_time=300)
def get_gpt_summary_of_taste(prompt):
    completion =  openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
    {"role": "assistant", "content": f"You are a movie, book, and tv expert. A user likes the following concepts in their taste profile: {prompt}. Can you summarize their taste as 5 different plot points? Do not output anything else, just the 5 phrases as a python list of strings"},
    ])
    return completion.choices[0].message["content"]


In [13]:
get_gpt_summary_of_taste(keywords)

"['mystery and crime-solving', 'drama and suspense', 'fast-paced action and thrillers', 'character-driven narratives', 'intricate world-building and philosophical themes']"